In [9]:
# Authors: Daelon Kingore, Ashton Barnwell, Jacob Albrecht
# Stock Market prediction using Data Mining/AI algorithms
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# gather data from csv
data = pd.read_csv('ORLY.csv')

# drop unnecessary data (can maybe use it in some models but not now)
data = data.drop(["Adj Close"], axis=1)
data = data.drop(["Date"], axis=1)
data = data.drop(["Volume"], axis=1)

# standardize data to allow more efficient and accurate processing
standardizer = StandardScaler()
dataStandardized = standardizer.fit_transform(data)

# find y by seeing if next day is up or down
y = np.where(data["Close"].shift(-1) > data["Close"], "Up", "Down")

# using the train test split function
xTrain, xTest, yTrain, yTest = train_test_split(dataStandardized, y, test_size=0.25, shuffle=False)



['Down' 'Down' 'Down' ... 'Up' 'Down' 'Down']
